In [1]:
from pyhealth.datasets import MIMIC4Dataset
mimic4_base = MIMIC4Dataset(
    root="/home/peter/files/PyHealth/data/mimic-iv2.2",
    tables=["diagnoses_icd", "procedures_icd", "prescriptions"],
    code_mapping={"NDC": ("ATC", {"target_kwargs": {"level": 3}})},
)
from pyhealth.tasks import mortality_prediction_mimic4_fn
mimic4_sample = mimic4_base.set_task(mortality_prediction_mimic4_fn)
mimic4_sample.samples[0]


Generating samples for mortality_prediction_mimic4_fn: 100%|██████████| 180733/180733 [00:05<00:00, 34837.12it/s]


{'visit_id': '22595853',
 'patient_id': '10000032',
 'conditions': [['5723',
   '78959',
   '5715',
   '07070',
   '496',
   '29680',
   '30981',
   'V1582']],
 'procedures': [['5491']],
 'drugs': [['B01A',
   'J07B',
   'A12B',
   'C03D',
   'C03C',
   'N02B',
   'J05A',
   'R03A',
   'N07B',
   'R03B']],
 'age': 51,
 'gender': 'F',
 'ethnicity': 'WHITE',
 'label': 0}

In [2]:
len(mimic4_sample.samples)

98772

In [3]:
import json
with open('/home/peter/files/PyHealth/nlp_task/mortality_pred/mimic4/mortality_pred_data.json', 'w') as f:
    json.dump(mimic4_sample.samples, f, indent=2)

In [3]:
from pyhealth.datasets import eICUDataset
eicu_base = eICUDataset(
    root="/home/peter/files/PyHealth/data/eicu",
    tables=["diagnosis", "medication", "physicalExam"],
    code_mapping={"NDC": ("ATC", {"target_kwargs": {"level": 3}})},
    # dev=True
)
from pyhealth.tasks import mortality_prediction_eicu_fn
eicu_sample = eicu_base.set_task(mortality_prediction_eicu_fn)
print(eicu_sample.samples[0])
print(len(eicu_sample.samples)) 

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


Parsing patients: 100%|██████████| 166355/166355 [01:29<00:00, 1853.98it/s]


finish basic patient information parsing : 91.6317708492279s
finish parsing diagnosis : 84.15019536018372s
finish parsing medication : 86.4191403388977s
finish parsing physicalExam : 166.36131262779236s


Mapping codes: 100%|██████████| 166355/166355 [00:08<00:00, 19561.95it/s]
Generating samples for mortality_prediction_eicu_fn:  15%|█▍        | 24859/166355 [00:01<00:07, 19306.85it/s]


TypeError: '<' not supported between instances of 'NoneType' and 'int'

In [2]:
dic_code_to_name = {}
import csv
import json
with open('/home/peter/files/PyHealth/data/eicu/diagnosis.csv', 'r') as f:
    reader = csv.reader(f)
    count = 0
    for row in reader:
        row50 = row[5].split(",")[0]
        if len(row[5].split(",")) > 1:
            row51 = row[5].split(",")[1]
            dic_code_to_name[row51] = row[4]
        dic_code_to_name[row50] = row[4]
        count += 1


for sample in eicu_sample.samples:
    for i in range(len(sample['conditions'][0])):
        if sample['conditions'][0][i] in dic_code_to_name:
            sample['conditions'][0][i] = dic_code_to_name[sample['conditions'][0][i]]
        else:
            sample['conditions'][0][i] = ""



with open('/home/peter/files/PyHealth/nlp_task/mortality_pred/eicu/mortality_pred_data.json', 'w') as f:
    json.dump(eicu_sample.samples, f, indent=2)